# Escuela Politécnica Nacional

### Objetivos

Este proyecto tiene como objetivo principal, medir el tiempo promedio que los estudiantes de la EPN se tardaron en culminar su carrera y encontrar si existe una relación entre la carrera y el tiempo de graduación

Librerías utilizadas

In [4]:
import tabula
import pandas as pd

### Parámetros y configuraciones

In [96]:
FILE_PATH_STUDENT = "inputs/APROBADOS INGENIERIA_2012.pdf"
FILE_PATH_STUDENT_HTML = "inputs/planesTesis.html"
FILE_PATH_STUDENTS_CSV = "epnStudents2021.csv"

## Extracción

Método para crear un archivo csv a partir del pdf

In [97]:
# TODO: Hacer que lea todas las página, de momento solo lee uno

In [98]:
def getInfoFromPdf():
    tabula.convert_into(FILE_PATH_STUDENT, "epnStudents2021.csv", pages='all')

Método para generar el dataset a partir un CSV

In [99]:
def getDatasetFromCSVFile(file_path_param):
    data_set = pd.read_csv(FILE_PATH_STUDENTS_CSV)
    return data_set

Método para generar el correo electrónico de los estudiantes, este campo será utilizado para hacer match
con el otro dataset

In [100]:
def addEmail(df):
    names = df['NOMBRES']
    apellidos = df['APELLIDOS']

    # Generar los correos electrónicos
    correos = []
    for nombre, apellido in zip(names, apellidos):
        nombre_partes = nombre.lower().split()
        apellido_partes = apellido.lower().split()

        # Construir el correo electrónico
        correo = f"{nombre_partes[0]}.{apellido_partes[0]}@epn.edu.ec.com"
        correos.append(correo)

    dataset['email'] = correos
    return dataset

Método para leer informacion de html

In [101]:
# Leer el archivo HTML y obtener una lista de DataFrames
dataframes = pd.read_html(FILE_PATH_STUDENT_HTML)

# Acceder al primer DataFrame de la lista
columnEmail = dataframes[0]
columnsName = ['Carrera','FecGradoOral','Email']
new_emp_list = []

for i in range(len(columnEmail)):
    carrera = columnEmail.iloc[i]['Carrera']
    fecGradoOral = columnEmail.iloc[i]['FecGradoOral']
    correos = columnEmail.iloc[i]['Email']
    for correo in correos.split():
        if "@epn.edu.ec" in correo:
            #print(correo)
            new_emp_list.append([carrera, fecGradoOral, correo])
dfTarget = pd.DataFrame(new_emp_list, columns=columnsName)
# Imprimir el nuevo DataFrame
print(dfTarget)

                                                Carrera        FecGradoOral  \
0                    (RRA) ELECTRONICA Y AUTOMATIZACION                 NaN   
1                    (RRA) ELECTRONICA Y AUTOMATIZACION                 NaN   
2     (RRA) MAESTRIA DE INVESTIGACION EN GESTION DE ...                 NaN   
3     (RRA) MAESTRIA DE INVESTIGACION EN GESTION DE ...                 NaN   
4     (RRA) MAESTRIA DE INVESTIGACION EN GESTION DE ...                 NaN   
...                                                 ...                 ...   
8385             TECNOLOGIA EN MANTENIMIENTO INDUSTRIAL  17/08/2018 0:00:00   
8386             TECNOLOGIA EN MANTENIMIENTO INDUSTRIAL                 NaN   
8387             TECNOLOGIA EN MANTENIMIENTO INDUSTRIAL                 NaN   
8388      TECNOLOGIA EN PROCESOS DE PRODUCCION MECANICA  15/06/2017 0:00:00   
8389      TECNOLOGIA EN PROCESOS DE PRODUCCION MECANICA  15/06/2017 0:00:00   

                           Email  
0     juan.zambr

## Transformación

## Load

In [22]:
if __name__ == '__main__':
    getInfoFromPdf()
    dataset = getDatasetFromCSVFile(FILE_PATH_STUDENTS_CSV)
    dataset = addEmail(dataset)
    print(dataset)

            APELLIDOS               NOMBRES                    CARRERA  \
0       ABARCA ROBLES           JUAN DANIEL  INGENIERIA AGROINDUSTRIAL   
1     ACOSTA CEVALLOS        STALIN ROBERTO                 MATEMATICA   
2      ACOSTA ORDOÑEZ  CHRISTOPHER LEONARDO        INGENIERIA MECANICA   
3       ACOSTA ROSERO           LUIS FELIPE        INGENIERIA MECANICA   
4       ACUÑA MONTERO         MARLON ANDRES                     FISICA   
..                ...                   ...                        ...   
768    ZAPATA HERRERA      CHRISTIAN JAVIER        INGENIERIA MECANICA   
769    ZAPATA HIDALGO        JOSSELYN PAOLA  INGENIERIA AGROINDUSTRIAL   
770      ZAPATA TERAN     STEPHANY CAROLINA     INGENIERIA EMPRESARIAL   
771  ZURITA FERNANDEZ         DIEGO ESTEBAN    INGENIERIA EN PETROLEOS   
772       ZURITA YUGS          KEVIN JAVIER        INGENIERIA MECANICA   

                                 email  
0           juan.abarca@epn.edu.ec.com  
1         stalin.acosta@epn.e